In [10]:
# ========================
# 1. Download & Extract Dataset
# ========================
# The following lines are commented out because they require Kaggle authentication
# and the 'kaggle.json' file was not found.
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle kernels pull prtksrmwr/cats-v-dogs-classifiction-using-cnn

# The following lines are commented out because the zip file was not found.
# import zipfile
# zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
# zip_ref.extractall('/content')
# zip_ref.close()

# TODO: Add code here to load your dataset.
# You can upload the dataset manually or use alternative methods to download it.
# For example, if you have the dataset in a zip file named 'dogs-vs-cats.zip'
# uploaded to your Colab environment, you can uncomment the zipfile lines above
# and adjust the path if necessary.
# If your data is already extracted in '/content/train' and '/content/test',
# you can proceed to the Data Preprocessing section.


# ========================
# 2. Import Libraries
# ========================
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# ========================
# 3. Data Preprocessing
# ========================
# Ensure your training and testing data are in the specified directories.
# If you have obtained the dataset and extracted it, these paths should be correct.
train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/train',
    labels='inferred',
    label_mode='int',
    batch_size=128,  # Batch size adjust kar sakte ho 64 bhi
    image_size=(128,128),
    shuffle=True
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='/content/test',
    labels='inferred',
    label_mode='int',
    batch_size=128,
    image_size=(128,128),
    shuffle=True
)

# Normalize images
def process(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

# ========================
# 4. Create CNN Model
# ========================
model = Sequential([
    Conv2D(32,(3,3),activation='relu',padding='same',input_shape=(128,128,3)),
    MaxPool2D((2,2)),

    Conv2D(64,(3,3),activation='relu',padding='same'),
    MaxPool2D((2,2)),

    Conv2D(128,(3,3),activation='relu',padding='same'),
    MaxPool2D((2,2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.6),  # dropout slightly increased
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.summary()

# ========================
# 5. Compile Model
# ========================
optimizer = keras.optimizers.Adam(learning_rate=0.0005)  # reduced learning rate
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ========================
# 6. Callbacks
# ========================
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=4,  # slightly increased patience
    restore_best_weights=True
)

# ========================
# 7. Train Model on Full Dataset
# ========================
history = model.fit(
    train_ds,
    epochs=15,
    validation_data=validation_ds,
    callbacks=[early_stop]
)

# ========================
# 8. Plot Training History
# ========================
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc)+1)

plt.figure(figsize=(14,5))

plt.subplot(1,2,1)
plt.plot(epochs, acc, 'b-', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r-', label='Validation Accuracy')
plt.title('Training & Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

import cv2
import numpy as np

# 1. Read & convert to RGB
# Make sure '/content/16.webp' exists or replace with your test image path
test_image = cv2.imread('/content/16.webp')
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
plt.imshow(test_image)

# 2. Resize
test_image = cv2.resize(test_image, (128,128))

# 3. Normalize
test_input = test_image.astype('float32') / 255.0
test_input = np.expand_dims(test_input, axis=0)

# 4. Predict
prediction = model.predict(test_input)
print("Raw prediction:", prediction[0][0])

# 5. Human readable
if prediction[0][0] < 0.5:
    print("Prediction → cat 🐱")
else:
    print("Prediction → dog 🐶 ")

NotFoundError: Could not find directory /content/train